In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
ls ./..

Data/      Notebook/  README.md  SOW/


In [8]:
ls ./../Data

AUS.xlsx  IND.csv   USA.csv


In [23]:
df_aus = pd.read_excel('./../Data/AUS.xlsx')
df_aus.head()

,Unique ID,Patient Name,Vaccine Type,Date of Birth,Date of Vaccination
0,1,Mike,LMN,NaT,2022-05-11 00:00:00
1,2,Jonnathan,XYZ,1997-12-13,2021-13-13
2,3,Cristina,ABC,1998-03-12,2022-03-12 00:00:00


In [24]:
df_aus.isna().sum()

Unique ID              0
Patient Name           0
Vaccine Type           0
Date of Birth          1
Date of Vaccination    0
dtype: int64

In [25]:
df_ind = pd.read_csv('./../Data/IND.csv')
df_ind.head()

,ID,Name,DOB,VaccinationType,VaccinationDate,Free or Paid
0,1,Vikas,1998-12-01,XYZ,2022-01-01,F
1,2,Rahul,1982-08-13,ABC,2022-03-05,P
2,3,Sameer,1952-08-13,ABC,2022-02-20,F


In [26]:
df_ind.isna().sum()

ID                 0
Name               0
DOB                0
VaccinationType    0
VaccinationDate    0
Free or Paid       0
dtype: int64

In [27]:
df_usa = pd.read_csv('./../Data/USA.csv')
df_usa.head()

,ID,Name,VaccinationType,VaccinationDate
0,1,Sam,EFG,6152022
1,2,John,XYZ,1052022
2,3,Mike,ABC,12282021


In [28]:
df_usa.isna().sum()

ID                 0
Name               0
VaccinationType    0
VaccinationDate    0
dtype: int64